In [59]:
import numpy as np
import plotly.graph_objects as go
from scipy.signal import find_peaks

# utils and helper_files
from utils.get_raw_data import get_multiple_data_arrays
from helper_files.normalizations import normalize_high_peak
from helper_files.gaussian_fitting import gaussian
from helper_files.peak_ratios import split_arrs, channel_to_kev, find_peaks_arrs, remove_linear_bg, n_gaussians_and_m_order_bg, print_table, make_fit, find_nearest, fwhm, table_values_FWHM, plotly_table


In [60]:
ga = get_multiple_data_arrays(filters=['GaAs'], normalize=False)
# highest peak normalization
norm = normalize_high_peak(ga)
norm, names = split_arrs(norm)
s = norm[3]

In [61]:
dispersion = 0.0100283 # from GaAs, Ga La and As Ka peaks
offset = 21.079
deg = 12 # degree of the polynomial to the background
prominence = 0.01 # prominence of the peaks, for the find_peaks
pixel_removal = 50 # removal of pixels before and after each peak
channel_points = 2048
Vacc = 30

channels = np.arange(0, channel_points)
res_increase = 10 # resolution increase for fit
channels_high_res = np.linspace(0, len(channels), len(channels) * res_increase)
# to center the high_res on channels, we need to shift it by half res_increase
# channels_high_res -= res_increase / 20

keV = channel_to_kev(arr=channels, dispersion=dispersion, offset=offset)
keV_high_res = channel_to_kev(arr=channels_high_res, dispersion=dispersion, offset=offset)

title = 'GaAs'

# the keV of the peaks for the table
gaLa = 1.098
asLa = 1.282
gaKa = 9.251
asKa = 10.543
gaKb = 10.264
asKb = 11.726

# p_names = ["Ga La", "As La", "Ga Ka", "As Ka", "Ga Kb", "As Kb"]
poi = [gaLa, asLa, gaKa, asKa, gaKb, asKb] # poi, peak of interest

In [165]:
peaks, _ = find_peaks(s, prominence=prominence)
# fig = go.Figure()
# fig.update_layout(title="Plot on channels, find the calibration peaks")
# fig.add_trace(go.Scatter(x=channels, y=s, mode="markers", name=names[0]))
# for p in peaks:
#     fig.add_vline(x=p, line_width=1, line_dash="dash", line_color="red", annotation_text=f"{p}")
# fig

In [7]:
bg40 = remove_linear_bg(s, peaks, pixel_removal=40, offset=int(offset))
bg100 = remove_linear_bg(s, peaks, pixel_removal=100, offset=int(offset))

fig = go.Figure()
fig.update_layout(title="Plot on channels, find the calibration peaks")
# fig.add_trace(go.Scatter(x=channels, y=s, mode="lines", name='raw'))
fig.add_trace(go.Scatter(x=keV, y=bg40, mode="lines", name='bg'))
fig.add_trace(go.Scatter(x=keV, y=bg100, mode="lines", name='bg'))
for p in peaks:
    pk = channel_to_kev(p, dispersion=dispersion, offset=offset)
    # fig.add_vline(x=p, line_width=1, line_dash="dash", line_color="red", annotation_text=f"{p}")
    fig.add_vline(x=pk, line_width=1, line_dash="dash", line_color="grey")

fig.update_layout(title=title, xaxis_title='keV', yaxis_title='Normalized intensity')

'2'


'2'

In [30]:
fit_vals, _ = make_fit(s, channels, deg=deg)
fit_high_res = n_gaussians_and_m_order_bg(channels, deg, *fit_vals, high_res=True, res_increase=res_increase)
fig = go.Figure()
fig.add_trace(go.Scatter(x=keV, y=s, mode="lines", name='raw'))
fig.add_trace(go.Scatter(x=keV_high_res, y=fit_high_res, mode="lines", name='fit high res'))

for p in fit_vals[deg + 2::3]:
    p = channel_to_kev(p, dispersion=dispersion, offset=offset)
    fig.add_vline(x=p, line_width=1, line_dash="dash", annotation_text=f"{p:.1f}")



In [34]:
fit_vals[:deg+1]

array([ 7.61941376e-38, -5.30834547e-34,  1.31249197e-31,  1.02806676e-26,
       -4.72681188e-23,  1.07726058e-19, -1.47828722e-16,  1.28060126e-13,
       -6.97005491e-11,  2.28834562e-08, -4.22218372e-06,  4.04519071e-04,
       -4.20987153e-03])

In [166]:
fit_amp = fit_vals[deg+1::3]
fit_peaks = fit_vals[deg+2::3]
fit_std = fit_vals[deg+3::3]
fit_bg = np.polyval(fit_vals[:deg+1], channels)

fit_high_res = n_gaussians_and_m_order_bg(channels, deg, *fit_vals, high_res=True, res_increase=res_increase)
fit = n_gaussians_and_m_order_bg(channels, deg, *fit_vals)

peaks_keV = channel_to_kev(fit_peaks, dispersion=dispersion, offset=offset)
std_keV = channel_to_kev(fit_std, dispersion=dispersion, offset=offset, use_offset=False)

fig = go.Figure()
fig.update_layout(title=f"Peak fit and backround fit of GaAs 30 kV")
fig.add_trace(go.Scatter(x=keV, y=s, mode="lines", name='raw'))
fig.add_trace(go.Scatter(x=keV_high_res, y=fit_high_res, mode="lines", name = 'fit high res'))
fig.add_trace(go.Scatter(x=keV, y=fit, mode="lines", name = 'fit'))
fig.add_trace(go.Scatter(x=keV, y=fit_bg, name='bg', mode="lines"))
fig.add_trace(go.Scatter(x=keV, y=fit-fit_bg, mode="lines", name = 'fit - bg'))
fig.add_trace(go.Scatter(x=keV, y=s-fit_bg, mode="lines", name = 'raw - bg'))

# add each gaussian to the plot
peaks_high_res = []
for i in range(len(fit_peaks)):
    p_hr = gaussian(keV, fit_amp[i], peaks_keV[i], std_keV[i], high_res=True, res_increase=res_increase)
    fig.add_trace(go.Scatter(x=keV_high_res, y=p_hr, name=f"{peaks_keV[i]:.1f} keV"))
    # fig.add_trace(go.Scatter(x=keV, y=gaussian(keV, fit_amp[i], peaks_keV[i], std_keV[i]), name=f"{peaks_keV[i]:.1f} keV"))
    peaks_high_res.append(p_hr)
for p in peaks_keV:
    fig.add_vline(x=p, line_width=1, line_dash="dash", line_color="grey", annotation_text=f"{p:.3f}")

# fig
# fig.write_html('../plots/final_fit_GaAs.html')

In [11]:
# # plot with new background color for presentation
# fig = go.Figure()
# fig.update_layout(title=f"Peak fit and backround fit of GaAs 30 kV")
# fig.add_trace(go.Scatter(x=keV, y=s, mode="markers", name='raw', marker=dict(
#             color='LightSkyBlue',
#             size=4,
#         ),))
# fig.add_trace(go.Scatter(x=keV_high_res, y=fit_high_res, mode="lines", name = 'fit'))
# fig.add_trace(go.Scatter(x=keV, y=fit_bg, name='background', mode="lines"))
# fig.update_layout(paper_bgcolor='#018576', plot_bgcolor='#018576', font_color='#fee7a6', font_size=14, xaxis_title="Energy [keV]", yaxis_title="Normalized counts")
# # set x max to 12.1 and y max to 0.5
# fig.update_xaxes(range=[0, 12.1])
# fig.update_yaxes(range=[0, 0.5])
# # fig.show()
# # fig.write_image('../plots/final_fit_GaAs__presentation.svg', width=1000, height=500, scale=6)
# # fig.write_html('../plots/final_fit_GaAs.html')

In [12]:
def make_table_ratios(peaks_keV, peaks_high_res, Vacc, stds_keV, poi):
    poi_idx_hr = [find_nearest(peaks_keV, p) for p in poi]
    p_values_hr = peaks_keV[poi_idx_hr]
    p_area = [sum(peaks_high_res[i]) for i in poi_idx_hr]
    p_fwhm = [fwhm(stds_keV[i])*1000 for i in poi_idx_hr]

    # ratio of la, ka, kb and la+ka and la+ka+kb
    la = p_area[0] / p_area[1]
    ka = p_area[2] / p_area[3]
    kb = p_area[4] / p_area[5]
    la_ka = (p_area[0] + p_area[2]) / (p_area[1] + p_area[3])
    la_ka_kb = (p_area[0] + p_area[2] + p_area[4]) / (p_area[1] + p_area[3] + p_area[5])

    header = ["Peak", "Ratio", "Ga value [keV]", "As value [keV]", "Ga FWHM [eV]", "As FWHM [eV]", "Ga sum", "As sum"]
    # adjust to 3 decimals
    row1 = ['La', la, p_values_hr[0], p_values_hr[1], p_fwhm[0], p_fwhm[1], p_area[0], p_area[1]]
    row2 = ['Ka', ka, p_values_hr[2], p_values_hr[3], p_fwhm[2], p_fwhm[3], p_area[2], p_area[3]]
    row3 = ['Kb', kb, p_values_hr[4], p_values_hr[5], p_fwhm[4], p_fwhm[5], p_area[4], p_area[5]]
    row4 = ['La+Ka', la_ka, '-', '-', '-', '-', p_area[0]+p_area[2], p_area[1]+p_area[3]]
    row5 = ['La+Ka+Kb', la_ka_kb, '-', '-', '-', '-', p_area[0]+p_area[2]+p_area[4], p_area[1]+p_area[3]+p_area[5]]

    # if Vacc is below 15, we only see La
    if Vacc < 15:
        table = np.array([row1]).T
        return header, table
    else:
        table = np.array([row1, row2, row3, row4, row5]).T
        return header, table



In [13]:
header, table = make_table_ratios(peaks_keV, peaks_high_res, Vacc, std_keV, poi)
# plotly_table(header, table, Vacc)

In [14]:
factors = [3.268, 4.191] # k-factors for Ga and As from AZtec
# factors[0] / factors[1] #  0.780

# 1.678 *  factors[0] / factors[1] # 1.308
# factors[1] / factors[0] # 1.282

In [15]:
print_table(header, table)

Peak             & Ratio            & Ga value [keV]   & As value [keV]   & Ga FWHM [eV]     & As FWHM [eV]     & Ga sum           & As sum           & 
La               & 2.279            & 1.098            & 1.287            & 72.309           & 80.849           & 76.465           & 33.546           & 
Ka               & 1.678            & 9.253            & 10.542           & 157.799          & 168.238          & 58.718           & 34.994           & 
Kb               & 1.603            & 10.276           & 11.736           & 171.804          & 185.034          & 8.821            & 5.503            & 
La+Ka            & 1.972            & -                & -                & -                & -                & 135.184          & 68.540           & 
La+Ka+Kb         & 1.945            & -                & -                & -                & -                & 144.004          & 74.042           & 


In [16]:
def make_final_table(data, Vacc, channels=channels, dispersion=dispersion, offset=offset, prominence=prominence, deg=deg, res_increase=res_increase, keV=keV, keV_high_res=keV_high_res, poi=None, plotlyplot=False, plotlytable=False):
    # does fitting and makes a table for one spectrum
    print(f'Finding the peaks from {Vacc} kV')
    fit_vals, _ = make_fit(data, channels, deg=deg, prominence=prominence, pixel_removal=pixel_removal, offset=offset)
    
    # using the fit_vals to make curves
    fit_hr = n_gaussians_and_m_order_bg(channels, deg, *fit_vals, high_res=True, res_increase=res_increase)
    fit_bg = np.polyval(fit_vals[:deg+1], channels)

    # making the standalone peaks
    fit_amp = fit_vals[deg+1::3]
    fit_peaks = fit_vals[deg+2::3]
    fit_std = fit_vals[deg+3::3]
    peaks_keV = channel_to_kev(fit_peaks, dispersion=dispersion, offset=offset)
    std_keV = channel_to_kev(fit_std, dispersion=dispersion, offset=offset, use_offset=False)
    peaks_high_res = []
    for i in range(len(fit_peaks)):
        peaks_high_res.append(gaussian(keV, fit_amp[i], peaks_keV[i], std_keV[i], high_res=True, res_increase=res_increase))

    # optional plotting
    if plotlyplot:
        fig = go.Figure()
        fig.update_layout(title=f"{Vacc} kV, peak fit and backround fit")
        fig.add_trace(go.Scatter(x=keV, y=data, mode="lines", name='raw'))
        fig.add_trace(go.Scatter(x=keV_high_res, y=fit_hr, mode="lines", name = 'fit high res'))
        fig.add_trace(go.Scatter(x=keV, y=fit_bg, name='bg', mode="lines"))
        for i in range(len(fit_peaks)):
            p_hr = gaussian(keV, fit_amp[i], peaks_keV[i], std_keV[i], high_res=True, res_increase=res_increase)
            fig.add_trace(go.Scatter(x=keV_high_res, y=p_hr, name=f"{peaks_keV[i]:.1f} keV", mode="lines"))
        for p in peaks_keV:
            fig.add_vline(x=p, line_width=1, line_dash="dash", line_color="grey", annotation_text=f"{p:.3f}")
        fig.show()
    
    # making the table, either with ratios or just values and FWHM
    if poi is not None:
        header, table = make_table_ratios(peaks_keV, peaks_high_res, Vacc, std_keV, poi)
    else:
        header, table = table_values_FWHM(peaks_keV, peaks_high_res, std_keV)
    # optional table
    if plotlytable:
        plotly_table(header, table, Vacc)
    
    # returning a dict with the essential data
    return_dict = {
        'Vacc': Vacc,
        'fit_vals': fit_vals,
        'fit_hr': fit_hr,
        'fit_bg': fit_bg,
        'fit_amp': fit_amp,
        'fit_peaks': fit_peaks,
        'fit_std': fit_std,
        'peaks_keV': peaks_keV,
        'std_keV': std_keV,
        'header': header,
        'table': table
    }
    return return_dict
    

In [17]:
s_dicts = []
Vaccs = [5, 10, 15, 30]
for s in range(len(norm)):
    s_dicts.append(make_final_table(norm[s], Vaccs[s], plotlyplot=False, plotlytable=False, poi=poi))


Finding the peaks from 5 kV
Finding the peaks from 10 kV
Finding the peaks from 15 kV
Finding the peaks from 30 kV


In [18]:
# print all tables
for s in s_dicts:
    print(f"\n{s['Vacc']} kV")
    print_table(s['header'], s['table'], delim='| ')


5 kV
Peak             |  Ratio            |  Ga value [keV]   |  As value [keV]   |  Ga FWHM [eV]     |  As FWHM [eV]     |  Ga sum           |  As sum           |  
La               |  1.282            |  1.101            |  1.288            |  74.010           |  80.921           |  75.462           |  58.844           |  

10 kV
Peak             |  Ratio            |  Ga value [keV]   |  As value [keV]   |  Ga FWHM [eV]     |  As FWHM [eV]     |  Ga sum           |  As sum           |  
La               |  1.444            |  1.100            |  1.287            |  73.841           |  80.827           |  76.222           |  52.770           |  

15 kV
Peak             |  Ratio            |  Ga value [keV]   |  As value [keV]   |  Ga FWHM [eV]     |  As FWHM [eV]     |  Ga sum           |  As sum           |  
La               |  1.669            |  1.100            |  1.287            |  73.830           |  81.137           |  77.001           |  46.146           |  
Ka            

In [19]:
s = get_multiple_data_arrays(filters=['NW'], normalize=False)
# highest peak normalization
s = normalize_high_peak(s)
s, names = split_arrs(s)
s_dicts_NW = []
Vaccs = [5, 10, 15, 30]
plottt = False
for i in range(len(s)):
    # if Vaccs[i] == 30:
    #     plottt = True
    s_dicts_NW.append(make_final_table(s[i], Vaccs[i], plotlyplot=plottt, plotlytable=False, poi=poi))

# for s in s_dicts_NW:
#     print(f"\n{s['Vacc']} kV")
#     print_table(s['header'], s['table'], delim='| ')

Finding the peaks from 5 kV
Finding the peaks from 10 kV
Finding the peaks from 15 kV
Finding the peaks from 30 kV


In [20]:
#
##  trying with dispersion and offset from .emsa file, ie. AZtec
#
disp_AZtec = 0.01
offs_AZtec = 20
keV_AZtec = channel_to_kev(channels, dispersion=disp_AZtec, offset=offs_AZtec)
keV_AZtec_high_res = channel_to_kev(arr=np.linspace(0, len(channels), len(channels) * res_increase), dispersion=dispersion, offset=offset)

s = get_multiple_data_arrays(filters=['Ga'], normalize=False)
# highest peak normalization
s = normalize_high_peak(s)
s, names = split_arrs(s)
s_dicts_AZtec = []
Vaccs = [5, 10, 15, 30]
plottt = False
for i in range(len(s)):
    # if Vaccs[i] == 30:
    #     plottt = True
    s_dicts_AZtec.append(make_final_table(s[i], Vaccs[i], plotlyplot=plottt, plotlytable=False, poi=poi, dispersion=disp_AZtec, offset=offs_AZtec, keV=keV_AZtec, keV_high_res=keV_AZtec_high_res))

# print('Dispersion and offset from AZtec')
# for s in s_dicts_AZtec:
#     print(f"\n{s['Vacc']} kV")
#     print_table(s['header'], s['table'], delim='| ')

Finding the peaks from 5 kV
Finding the peaks from 10 kV
Finding the peaks from 15 kV
Finding the peaks from 30 kV


In [23]:
s_dicts

[{'Vacc': 5,
  'fit_vals': array([-1.60501138e-37,  2.64162151e-33, -1.88160861e-29,  7.69253147e-26,
         -2.01049346e-22,  3.53155181e-19, -4.25825888e-16,  3.52980489e-13,
         -1.97794122e-10,  7.18881537e-08, -1.54959053e-05,  1.56636526e-03,
         -1.50166022e-02,  5.03441389e-02,  2.17602962e+01,  1.44734478e+00,
          2.23839916e-02,  4.59115661e+01,  1.52995287e+00,  4.71879788e-02,
          7.17181713e+01,  2.72926829e+00,  6.60224634e-02,  1.16688645e+02,
          3.82231391e+00,  9.60147823e-01,  1.30893184e+02,  3.13405766e+00,
          6.84770336e-01,  1.49465864e+02,  3.42670746e+00]),
  'fit_hr': array([-0.01423763, -0.01408275, -0.01392819, ...,  0.00039502,
          0.00039799,  0.00040097]),
  'fit_bg': array([-0.0150166 , -0.01346566, -0.01194528, ...,  0.00034327,
          0.0003716 ,  0.00040097]),
  'fit_amp': array([0.05034414, 0.02238399, 0.04718798, 0.06602246, 0.96014782,
         0.68477034]),
  'fit_peaks': array([ 21.76029616,  45.91156

In [22]:
print('Dispersion and offset from calibration of Ga La and As Ka')

# kf5 = [1.210, 1.086] # 5kV k-factors, As then Ga on the L-series
# kf10 = [1.310, 1.223] # 10kV k-factors, Ga and As L-series
# kf15 = [1.331, 1.259] # 15kV k-factors, Ga and As L-series
# kf30 = [4.191, 3.268] # 30kV k-factors, Ga and As K-series
# factors = [kf5, kf10, kf15, kf30] # [As, Ga]
# ratios = [kf5[1]/kf5[0], kf10[1]/kf10[0], kf15[1]/kf15[0], kf30[1]/kf30[0]] # Ga/As ratios

for s in s_dicts:
    print(f"\n{s['Vacc']} kV")
    print_table(s['header'], s['table'], delim='& ')

Dispersion and offset from calibration of Ga La and As Ka

5 kV
Peak             &  Ratio            &  Ga value [keV]   &  As value [keV]   &  Ga FWHM [eV]     &  As FWHM [eV]     &  Ga sum           &  As sum           &  
La               &  1.282            &  1.101            &  1.288            &  74.010           &  80.921           &  75.462           &  58.844           &  

10 kV
Peak             &  Ratio            &  Ga value [keV]   &  As value [keV]   &  Ga FWHM [eV]     &  As FWHM [eV]     &  Ga sum           &  As sum           &  
La               &  1.444            &  1.100            &  1.287            &  73.841           &  80.827           &  76.222           &  52.770           &  

15 kV
Peak             &  Ratio            &  Ga value [keV]   &  As value [keV]   &  Ga FWHM [eV]     &  As FWHM [eV]     &  Ga sum           &  As sum           &  
La               &  1.669            &  1.100            &  1.287            &  73.830           &  81.137           